In [ ]:
# MNIST preparation p2
import os
import torch
import torch.utils.data as data
import json
from PIL import Image
import torchvision.transforms.v2 as tfs
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm



class DigitDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path, "train" if train else "test")
        self.transform = transform

        with open(os.path.join(path, "format.json"), "r") as fp:
            self.format = json.load(fp)

        self.length = 0
        self.files = []
        self.targets = torch.eye(10)

        for _dir, _target in self.format.items():
            path = os.path.join(self.path, _dir)
            list_files = os.listdir(path)
            self.length += len(list_files)
            self.files.extend(map(lambda _x: (os.path.join(path, _x), _target), list_files))

    def __getitem__(self, item):
        path_file, target = self.files[item]
        t = self.targets[target]
        img = Image.open(path_file)

        if self.transform:
            img = self.transform(img).ravel().float() / 255.0

        return img, t

    def __len__(self):
        return self.length


class DigitNN(nn.Module):
    def __init__(self, input_dim, num_hid, out_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim,num_hid)
        self.layer2 = nn.Linear(num_hid, out_dim)

    def forward(self,x):
        x = self.layer1(x)
        x = nn.functional.relu(x)
        x = self.layer2(x)
        return x

model = DigitNN(28 * 28, 32, 10)

# Setting trnsform
to_tensor = tfs.ToImage()
d_dataset = DigitDataset("datasets/MNIST", transform=to_tensor)
train_data = data.DataLoader(d_dataset, batch_size=32, shuffle=True)

optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()

ep = 2
model.train()

for _e in range(ep):
    train_tqdm = tqdm(train_data, leave=True)

    for x_train_img, y_target in train_tqdm:
        predict = model(x_train_img)
        loss = loss_function(predict, y_target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Test
d_test = DigitDataset("datasets/MNIST", train=False, transform=to_tensor)
test_data = data.DataLoader(d_test , batch_size=500, shuffle=False)
Q = 0

model.eval()

for x_test, y_test in test_data:
    p = model(x_test) # 500, 10
    p = torch.argmax(p, dim=1) # 500,1
    y = torch.argmax(y_test, dim=1) # 500,1
    Q += torch.sum(p == y).item()

Q /= len(d_test)
print(Q)



In [ ]:
# 3
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim

from sklearn.datasets import load_iris
import torch
import tqdm

iris = load_iris()
_global_var_data_x = torch.tensor(iris.data, dtype=torch.float32)
_global_var_target = torch.tensor(iris.target, dtype=torch.long)

class IrisDataset(data.Dataset):
    def __init__(self):
        self.data = _global_var_data_x # тензор размерностью (150, 4), тип float32
        self.target = _global_var_target # тензор размерностью (150, ), тип int64 (long)

        self.length = len(self.target) # размер выборки
        self.categories = ['setosa' 'versicolor' 'virginica'] # названия классов
        self.features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

    def __getitem__(self, item):
        return self.data[item], self.target[item],
        # возврат образа по индексу item в виде кортежа: (данные, целевое значение)

    def __len__(self):
        # возврат размера выборки
        return self.length


class IrisClassModel(nn.Module):
    def __init__(self, in_features=4, out_features=3):
        super().__init__()
        # модель нейронной сети из двух полносвязных слоев:
        # 1-й слой: число входов in_features, число нейронов 16
        # 2-й слой: число нейронов out_features
        self.layer1 = nn.Linear(in_features, 16)
        self.layer2 = nn.Linear(16, out_features)

    def forward(self, x):
        # тензор x пропускается через 1-й слой
        x = self.layer1(x)
        # через функцию активации torch.relu()
        x = nn.functional.relu(x)
        # через второй слой
        x = self.layer2(x)
        # полученный (вычисленный) тензор возвращается
        return x


torch.manual_seed(11)

# создать модель IrisClassModel с числом входов 4 и числом выходов 3
model = IrisClassModel(4,3)
# перевести модель в режим обучения
model.train()

epochs = 10 # число эпох обучения
batch_size = 8 # размер батча

# создать объект класса IrisDataset
dataset_train = IrisDataset()
train_data = data.DataLoader(dataset_train, batch_size=batch_size,shuffle=True) # создать объект класса DataLoader с размером пакетов batch_size и перемешиванием образов выборки

optimizer = optim.Adam(params=model.parameters(), lr = 0.01)# создать оптимизатор Adam для обучения модели с шагом обучения 0.01
loss_func = nn.CrossEntropyLoss() # создать функцию потерь с помощью класса CrossEntropyLoss (используется при многоклассовой классификации)

for _e in range(epochs): # итерации по эпохам
    for x_train, y_train in train_data:
        predict = model(x_train) # вычислить прогноз модели для данных x_train
        loss = loss_func(predict, y_train) # вычислить значение функции потерь

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# перевести модель в режим эксплуатации
model.eval()

# выполнить прогноз модели по всем данным выборки
predict = model(dataset_train.data)
p = torch.argmax(predict, dim=1)


Q = torch.mean((p == dataset_train.target).float()).item() # вычислить долю верных классификаций (сохранить, как вещественное число, а не тензор)

In [ ]:
# 4

from sklearn.datasets import load_wine
import torch


wine_data = load_wine()

# Признаки и целевые значения
_global_var_data_x = torch.tensor(wine_data.data, dtype=torch.float32)
_global_var_target = torch.tensor(wine_data.target, dtype=torch.long)

# Размеры и типы
print("Размеры и типы:", _global_var_data_x.shape, _global_var_data_x.dtype, _global_var_target.shape, _global_var_target.dtype)

# Прочие характеристики
print("Количество классов: ", len(wine_data.target_names))
print("Названия классов: ", wine_data.target_names)
print("Названия признаков: ", wine_data.feature_names)


import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim


class WineDataset(data.Dataset):
    def __init__(self):
        self.data = _global_var_data_x # тензор размерностью (178, 13), тип float32
        self.target = _global_var_target # тензор размерностью (178, ), тип int64 (long)

        self.length = len(self.target) # размер выборки
        self.categories = ['class_0', 'class_1', 'class_2'] # названия классов

    def __getitem__(self, item):
        return self.data[item], self.target[item]
        # возврат образа по индексу item в виде кортежа: (данные, целевое значение)

    def __len__(self):
        return self.length
        # возврат размера выборки

# здесь продолжайте программу
class WineClassModel(nn.Module):
    def __init__(self, in_features=13, out_features=3):
        super().__init__()
        self.layer1 = nn.Linear(in_features, 32)
        self.layer2 = nn.Linear(32, 16)
        self.layer3 = nn.Linear(16, out_features)

    def forward(self, x):
        x = self.layer1(x).relu()
        x = self.layer2(x).relu()
        x = self.layer3(x)
        return x

dataset_train = WineDataset()
train_data = data.DataLoader(dataset_train, batch_size=16, shuffle=True)

model = WineClassModel()
optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_func = nn.CrossEntropyLoss()

for _e in range(20):
    for x_train, y_train in train_data:
        predict = model(x_train)
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    

model.eval()
predict_all = torch.argmax(model(dataset_train.data), dim=1)
tgt = dataset_train.target 
Q = torch.mean( (predict_all == dataset_train.target ).float()).item()


Размеры и типы: torch.Size([178, 13]) torch.float32 torch.Size([178]) torch.int64
Количество классов:  3
Названия классов:  ['class_0' 'class_1' 'class_2']
Названия признаков:  ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [ ]:
# 5
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim


# здесь продолжайте программу
from sklearn.datasets import load_digits
import torch


digits = load_digits()

# Признаки и целевые значения
_global_var_data_x = torch.tensor(digits.data, dtype=torch.float32)
_global_var_target = torch.tensor(digits.target, dtype=torch.int64)


class DigitsDataset(data.Dataset):
    def __init__(self):
        self.data = _global_var_data_x # тензор размерностью (1797, 64), тип float32
        self.target = _global_var_target # тензор размерностью (1797, ), тип int64 (long)

        self.length = len(self.target) # размер выборки

    def __getitem__(self, item):
        return self.data[item], self.target[item]
        # возврат образа по индексу item в виде кортежа: (данные, целевое значение)

    def __len__(self):
        return self.length
        # возврат размера выборки


# здесь продолжайте программу
class DigitsClassModel(nn.Module):
    def __init__(self, in_features=64, out_features=10):
        super().__init__()
        self.layer1 = nn.Linear(in_features, 32)
        self.layer2 = nn.Linear(32, 16)
        self.layer3 = nn.Linear(16, out_features)

    def forward(self, x):
        x = self.layer1(x).relu()
        x = self.layer2(x).relu()
        x = self.layer3(x)
        return x


dataset_train = DigitsDataset()
train_data = data.DataLoader(dataset_train, batch_size=12, shuffle=True)

model = WineClassModel()
optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_func = nn.CrossEntropyLoss()

for _e in range(10):
    for x_train, y_train in train_data:
        predict = model(x_train)
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    

model.eval()
predict_all = torch.argmax(model(dataset_train.data), dim=1)
tgt = dataset_train.target 
Q = torch.mean( (predict_all == dataset_train.target ).float()).item()



torch.Size([1797, 64]) torch.Size([1797])
